In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = todos los mensajes, 1 = INFO, 2 = WARNING, 3 = ERROR
import numpy as np
import keras
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from matplotlib import cm
from scipy.ndimage import label

In [2]:
from tensorflow.keras.models import Model
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from data import part_classes, animals

In [4]:
IMG_SIZE = (360,468)
total_pixels = IMG_SIZE[0]*IMG_SIZE[1]
pixel_region = round(total_pixels*0.02)

In [5]:
OPTION = 1
if OPTION == 1:
    model_name = 'vgg16_without_model'
    data_metrics_file = 'dataset_info_parts_metrics_methods_without'
else:
    model_name = 'vgg16_model'
    data_metrics_file = 'dataset_info_parts_metrics_methods'

In [6]:
model = tf.keras.models.load_model(model_name+'.h5')
conv_layers= [l.name for l in model.layers if l.__class__.__name__.endswith('Conv2D')]
dense_layers= [l.name for l in model.layers if l.__class__.__name__.endswith('Dense')]
last_conv_layer_name = conv_layers[-1]

In [7]:
df = pd.read_csv('dataset_info_parts.csv')
PATH_data = 'dataset/'
IMG_RESIZE_PATH_data = PATH_data+'images_resize/'
ANN_RESIZE_PATH_data = PATH_data+'annotations_resize/'
IMG_SIZE = (360,468)
BATCH_SIZE = 32
df['file_path'] = IMG_RESIZE_PATH_data + df['file']+'.jpg'
animals = ['sheep', 'dog', 'cow', 'cat', 'horse']

df[['pred_id', 'pred_ok', 'cat_id']] = np.nan
df[['mean_act_obj', 'mean_act_back', 'perc_act_obj', 'perc_act_back']] = np.nan

In [8]:
array = np.array([])
array_y = np.array([])
for animal in animals:
    df_a = df[df['cat']==animal]
    cat_id = animals.index(animal)
    count = 0
    for index, row in df_a.iterrows():
        img = cv2.cvtColor(cv2.imread(row.file_path),cv2.COLOR_BGR2RGB)
        if len(array)==0:
            array=np.expand_dims(img.astype(np.float32), axis=0)
        else:
            array = np.concatenate([array, [img]],axis=0)
        if len(array_y)==0:
            array_y = np.array([cat_id])
        else:
            array_y = np.concatenate([array_y, [cat_id]],axis=0)
        count += 1
        if count>10:
            break     
            

In [9]:
data = {}
all_stats = []
for conv_layer in conv_layers+dense_layers:
    intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(conv_layer).output)
    activations = intermediate_layer_model.predict(array,batch_size=2)  # (1000, h, w, c)
    if conv_layer.startswith('block'):
        n_samples, h, w, c = activations.shape
        flattened = activations.reshape(n_samples, -1)  # (1000, h*w*c)
        mean_per_filter = flattened.reshape(n_samples, h*w, c).mean(axis=1)
    else:
        n_samples, d = activations.shape
        mean_per_filter = activations.reshape(n_samples, -1)  # (1000, h*w*c)
        #mean_per_filter = flattened.reshape(n_samples, d).mean(axis=1)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(mean_per_filter)
    mi_scores = mutual_info_classif(X_scaled, array_y)
    data[conv_layer] = mi_scores
    """
    plt.figure(figsize=(12, 5))
    plt.bar(range(len(mi_scores)), mi_scores)
    plt.xlabel("Índice de filtro (canal)")
    plt.ylabel("MIS con clase")
    plt.title(f"Mutual Information Score - Capa: {conv_layer}")
    plt.show()
    """
    summary_stats = {
        'layer': conv_layer,
        'mean': np.mean(mi_scores),
        'std': np.std(mi_scores),
        'max': np.max(mi_scores),
        'min': np.min(mi_scores),
        'median': np.median(mi_scores)
    }
    print(summary_stats)
    all_stats.append(summary_stats)

df_stats = pd.DataFrame(all_stats)
df_stats.to_csv('mis_stats_'+str(OPTION)+'.csv', index=False)

28/28 [==============================] - 9s 11ms/step
{'layer': 'block1_conv1', 'mean': 0.049918904968625645, 'std': 0.055187797774577274, 'max': 0.20803059567481386, 'min': 0.0, 'median': 0.03325973407321903}
28/28 [==============================] - 1s 13ms/step
{'layer': 'block1_conv2', 'mean': 0.06691162766234414, 'std': 0.07104261345168872, 'max': 0.23826428376626874, 'min': 0.0, 'median': 0.045350634545170854}
28/28 [==============================] - 1s 16ms/step
{'layer': 'block2_conv1', 'mean': 0.08715941804097203, 'std': 0.0799422631487306, 'max': 0.32258698153499266, 'min': 0.0, 'median': 0.07451535140389542}
28/28 [==============================] - 1s 15ms/step
{'layer': 'block2_conv2', 'mean': 0.08155707549169677, 'std': 0.08195154658465481, 'max': 0.3101969799845108, 'min': 0.0, 'median': 0.07144664265023071}
28/28 [==============================] - 1s 18ms/step
{'layer': 'block3_conv1', 'mean': 0.07415506375329903, 'std': 0.07833544320283428, 'max': 0.30496204129686477, 'm